In [1]:
from google.cloud import storage
import pandas as pd ## for dataset and eda
import numpy as np ## for eda
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


# %matplotlib inline
print("All good")

All good


In [2]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
# from transformers import DistilBertModel, DistilBertTokenizer # do not forget to conda install transformers
from transformers import *
print("All packages are imported")

All packages are imported


In [3]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(f"GPU usage is set up: {device}")

GPU usage is set up: cuda


In [4]:
cuda.is_available()

True

In [5]:
bucket_name = "firstprojectdl"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

print(bucket)
print('Great, we now have access to our first bucket on google cloud storage where we put our data')

<Bucket: firstprojectdl>
Great, we now have access to our first bucket on google cloud storage where we put our data


In [6]:
from google.cloud import storage
import pandas as pd

bucket_name = "firstprojectdl"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)

# When you have your files in a subfolder of the bucket.
my_prefix = "data/hw2/" # the name of the subfolder
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')


for blob in blobs:
    print(blob.name)
    if(blob.name != my_prefix): # ignoring the subfolder itself 
        file_name = blob.name.replace(my_prefix, "")
        blob.download_to_filename(file_name) # download the file to the machine
        print(file_name)
print(f"we imported the {file_name} successfully")

data/hw2/
data/hw2/nodeid2paperid.csv
nodeid2paperid.csv
data/hw2/sample.csv
sample.csv
data/hw2/testData.csv
testData.csv
data/hw2/textData.csv
textData.csv
data/hw2/trainData.csv
trainData.csv
we imported the trainData.csv successfully


In [7]:
nodeid2paperid = pd.read_csv('nodeid2paperid.csv')
nodeid2paperid.head()
nodeid2paperid.rename(columns={'node idx': 'id'}, inplace=True)
nodeid2paperid.head()

,id,paper id
0,104447,630234
1,15858,803423
2,107156,1102481
3,82077,1810480
4,42436,2131697


In [8]:
test = pd.read_csv('testData.csv')
test.head()

,id
0,137832
1,137833
2,137834
3,137836
4,137837


In [9]:
test.shape

(13718, 1)

In [10]:
text = pd.read_csv('textData.csv')
text.head()

,paper id,title,abstract
0,630234,spreadsheets on the move an evaluation of mobi...,The power of mobile devices has increased dram...
1,803423,multi view metric learning for multi view vide...,Traditional methods on video summarization are...
2,1102481,big data analytics in future internet of things,Current research on Internet of Things (IoT) m...
3,1810480,cryptographic hardening of d sequences,This paper shows how a one-way mapping using m...
4,2131697,gesture based continuous authentication for we...,We study the feasibility of touch gesture beha...


In [11]:
text.shape

(73718, 3)

In [12]:
train = pd.read_csv('trainData.csv')
train.head()

,label,id
0,4,0
1,5,1
2,8,3
3,6,6
4,4,7


In [13]:
train.shape

(60000, 2)

In [14]:
sample = pd.read_csv('sample.csv')
sample.head()

,id,label
0,137832,0
1,137833,0
2,137834,0
3,137836,0
4,137837,0


In [15]:
sample.shape

(13718, 2)

In [16]:
## Let us merge text in train and test
# train
trainData = pd.merge(train, nodeid2paperid, on='id', how='inner')

In [17]:
trainData = pd.merge(trainData, text, on ='paper id', how='inner')

In [18]:
trainData.head()

,label,id,paper id,title,abstract
0,4,0,9657784,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes..."
1,5,1,39886162,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...
2,8,3,121432379,a promise theory perspective on data networks,Networking is undergoing a transformation thro...
3,6,6,1444859417,webvrgis based city bigdata 3d visualization a...,This paper shows the WEBVRGIS platform overlyi...
4,4,7,1483430697,information theoretic authentication and secre...,"In the splitting model, information theoretic ..."


In [19]:
trainData.shape

(60000, 5)

In [20]:
# test
testData = pd.merge(test, nodeid2paperid, on='id', how='inner')

In [21]:
testData.head()

,id,paper id
0,137832,2403725649
1,137833,2404740077
2,137834,2407125866
3,137836,2408327416
4,137837,2412021890


In [22]:
testData = pd.merge(testData, text, on ='paper id', how='inner')

In [23]:
testData.head()

,id,paper id,title,abstract
0,137832,2403725649,patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal..."
1,137833,2404740077,the unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...
2,137834,2407125866,end to end goal driven web navigation,We propose a goal-driven web navigation as a b...
3,137836,2408327416,complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...
4,137837,2412021890,a parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...


In [24]:
testData.shape

(13718, 4)

In [25]:
# # Removing unwanted columns and only leaving title and abstract of the scientific article and the category which will be the label
trainData = trainData[['id','title', 'abstract', 'label']]
labelListData = list(trainData['label'].unique())
trainData.head()

,id,title,abstract,label
0,0,evasion attacks against machine learning at te...,"In security-sensitive applications, the succes...",4
1,1,how hard is computing parity with noisy commun...,We show a tight lower bound of $\Omega(N \log\...,5
2,3,a promise theory perspective on data networks,Networking is undergoing a transformation thro...,8
3,6,webvrgis based city bigdata 3d visualization a...,This paper shows the WEBVRGIS platform overlyi...,6
4,7,information theoretic authentication and secre...,"In the splitting model, information theoretic ...",4


In [26]:
testData = testData[['id','title', 'abstract']]
testData.head()

,id,title,abstract
0,137832,patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal..."
1,137833,the unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...
2,137834,end to end goal driven web navigation,We propose a goal-driven web navigation as a b...
3,137836,complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...
4,137837,a parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...


In [27]:
# Add a comma at the end of the titles
trainData['title'] = trainData['title'].apply(lambda x: x.capitalize())
trainData['title'] = trainData['title'].apply(lambda x: x + '.' if x[:-1] != '.' else x)

In [28]:
## merge the title and abstract columns together 
trainData['description'] = trainData['title'].str.cat(trainData['abstract'],sep=" ")
trainData['description'] = trainData['description'].apply(lambda x: x[0:510])
## trainData['description'] = trainData['title']

In [29]:
## add a period at the end of the truncated 

trainData['description'] = trainData['description'].apply(lambda x: x + '.' if x[-1] != '.' else x)

# Remove stop words in description and title ## Actually mention why you may not want to remove stop words: https://stackoverflow.com/questions/63633534/is-it-necessary-to-do-stopwords-removal-stemming-lemmatization-for-text-classif

In [30]:
trainData = trainData[['description', 'label']]

In [31]:
trainData.isna().sum()

description    0
label          0
dtype: int64

In [32]:
len(labelListData) # 20 labels and not 40

20

In [33]:
labelListData

[4, 5, 8, 6, 3, 16, 19, 14, 10, 0, 2, 18, 9, 13, 11, 1, 7, 15, 17, 12]

In [34]:
# biggest text in column
trainData['description'].apply(lambda x: len(x)).max()

511

In [35]:
trainData

,description,label
0,Evasion attacks against machine learning at te...,4
1,How hard is computing parity with noisy commun...,5
2,A promise theory perspective on data networks....,8
3,Webvrgis based city bigdata 3d visualization a...,6
4,Information theoretic authentication and secre...,4
...,...,...
59995,Incentivizing users of data centers participat...,5
59996,Semantic change detection with hypermaps. Chan...,16
59997,Computing with polynomial ordinary differentia...,9
59998,On energy efficiency in wireless networks a ga...,8


In [36]:
testData['title'] = testData['title'].apply(lambda x: x.capitalize())
# Add a comma at the end of the titles
testData['title'] = testData['title'].apply(lambda x: x + '.' if x[:-1] != '.' else x)
testData['description'] = testData['title'].str.cat(testData['abstract'],sep=" ")
testData['description'] = testData['description'].apply(lambda x: x[0:510])

In [37]:
## add a period at the end of the truncated 

testData['description'] = testData['description'].apply(lambda x: x + '.' if x[-1] != '.' else x)

In [38]:
testData

,id,title,abstract,description
0,137832,Patchlift fast and exact computation of patch ...,"In this paper, we propose a fast algorithm cal...",Patchlift fast and exact computation of patch ...
1,137833,The unreasonable effectiveness of address clus...,Address clustering tries to construct the one-...,The unreasonable effectiveness of address clus...
2,137834,End to end goal driven web navigation.,We propose a goal-driven web navigation as a b...,End to end goal driven web navigation. We prop...
3,137836,Complexity measures for map reduce and compari...,The programming paradigm Map-Reduce and its ma...,Complexity measures for map reduce and compari...
4,137837,A parallel implementation of the ensemble kalm...,This paper discusses an efficient parallel imp...,A parallel implementation of the ensemble kalm...
...,...,...,...,...
13713,169336,Confidence guided stereo 3d object detection w...,Accurate and reliable 3D object detection is v...,Confidence guided stereo 3d object detection w...
13714,169338,Sentinet detecting localized universal attacks...,SentiNet is a novel detection framework for lo...,Sentinet detecting localized universal attacks...
13715,169340,Learning compositional rules via neural progra...,"Many aspects of human reasoning, including lan...",Learning compositional rules via neural progra...
13716,169341,Certified defenses for adversarial patches.,Adversarial patch attacks are among one of the...,Certified defenses for adversarial patches. Ad...


In [39]:
testData = testData[['description']]

In [40]:
testData

,description
0,Patchlift fast and exact computation of patch ...
1,The unreasonable effectiveness of address clus...
2,End to end goal driven web navigation. We prop...
3,Complexity measures for map reduce and compari...
4,A parallel implementation of the ensemble kalm...
...,...
13713,Confidence guided stereo 3d object detection w...
13714,Sentinet detecting localized universal attacks...
13715,Learning compositional rules via neural progra...
13716,Certified defenses for adversarial patches. Ad...


In [41]:
# biggest text in column
testData['description'].apply(lambda x: len(x)).max()

511

In [42]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 7
LEARNING_RATE = 3e-05
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')

In [43]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        description = str(self.data.description[index])
        description = " ".join(description.split())
        inputs = self.tokenizer.encode_plus(
            description,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.label[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [44]:
# Creating the dataset and dataloader for the neural network
train_size = 0.8
train_dataset= trainData.sample(frac=train_size,random_state=200)
val_dataset= trainData.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(train.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALIDATION Dataset: {}".format(val_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
validation_set = Triage(val_dataset, tokenizer, MAX_LEN)

FULL Dataset: (60000, 2)
TRAIN Dataset: (48000, 2)
VALIDATION Dataset: (12000, 2)


In [45]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **val_params)

In [46]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class textClassModel(torch.nn.Module):
    def __init__(self):
        super(textClassModel, self).__init__()
        # self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.l1 = AutoModel.from_pretrained('allenai/scibert_scivocab_cased')
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 20)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [47]:
model = textClassModel()
model.to(device)

textClassModel(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31116, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [48]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

In [49]:
# Function to calcuate the accuracy of the model

def calculate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [50]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calculate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%1000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 1000 steps: {loss_step}")
            print(f"Training Accuracy per 1000 steps: {accu_step}")
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
for epoch in tqdm(range(EPOCHS)):
    train(epoch)

  0%|          | 0/7 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Training Loss per 1000 steps: 2.8768184185028076
Training Accuracy per 1000 steps: 12.5
Training Loss per 1000 steps: 0.8847812138624362
Training Accuracy per 1000 steps: 75.41833166833167
Training Loss per 1000 steps: 0.7612626315920666
Training Accuracy per 1000 steps: 77.82358820589705


 14%|█▍        | 1/7 [1:20:48<8:04:51, 4848.63s/it]

The Total Accuracy for Epoch 0: 79.03958333333334
Training Loss Epoch: 0.7071367165545622
Training Accuracy Epoch: 79.03958333333334
Training Loss per 1000 steps: 0.2207719087600708
Training Accuracy per 1000 steps: 87.5
Training Loss per 1000 steps: 0.44006357000364765
Training Accuracy per 1000 steps: 86.51348651348651
Training Loss per 1000 steps: 0.4359848263828472
Training Accuracy per 1000 steps: 86.34745127436283


 29%|██▊       | 2/7 [2:41:52<6:44:25, 4853.10s/it]

...

The Total Accuracy for Epoch 3: 94.16875
Training Loss Epoch: 0.18651209712525207
Training Accuracy Epoch: 94.16875
Training Loss per 1000 steps: 0.28319603204727173
Training Accuracy per 1000 steps: 87.5
Training Loss per 1000 steps: 0.1098328665583641
Training Accuracy per 1000 steps: 96.7032967032967
Training Loss per 1000 steps: 0.11912686200528548
Training Accuracy per 1000 steps: 96.42991004497752


 71%|███████▏  | 5/7 [6:45:03<2:42:00, 4860.05s/it]

The Total Accuracy for Epoch 4: 96.22916666666667
Training Loss Epoch: 0.12372853906701009
Training Accuracy Epoch: 96.22916666666667
Training Loss per 1000 steps: 0.03270751237869263
Training Accuracy per 1000 steps: 100.0
Training Loss per 1000 steps: 0.0764327486762991
Training Accuracy per 1000 steps: 97.6461038961039
Training Loss per 1000 steps: 0.08620677994511117
Training Accuracy per 1000 steps: 97.33570714642678


 86%|████████▌ | 6/7 [8:06:07<1:21:01, 4861.25s/it]

The Total Accuracy for Epoch 5: 97.16666666666667
Training Loss Epoch: 0.09321606374531985
Training Accuracy Epoch: 97.16666666666667
Training Loss per 1000 steps: 0.021113157272338867
Training Accuracy per 1000 steps: 100.0
Training Loss per 1000 steps: 0.05617453954317472
Training Accuracy per 1000 steps: 98.3016983016983
Training Loss per 1000 steps: 0.06706241544695392
Training Accuracy per 1000 steps: 97.95727136431785


In [447]:
torch.save(model.state_dict(), 'textClassModel.pt')

In [ ]:
def valid(epoch):
    model.eval()
    tr_loss = 0
    nb_tr_steps = 0
    n_correct = 0 
    nb_tr_examples = 0
    with torch.no_grad():
        for _, data in enumerate(validation_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calculate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%1000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 1000 steps: {loss_step}")
                print(f"Validation Accuracy per 1000 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu


In [ ]:
EPOCHSVAL = 3
for epoch in tqdm(range(EPOCHSVAL)):
    acc = valid(epoch)
    print("Accuracy on test data = %0.2f%%" % acc)

In [450]:
class TriageTest(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        description = str(self.data.description[index])
        description = " ".join(description.split())
        inputs = self.tokenizer.encode_plus(
            description,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
        } 
    
    def __len__(self):
        return self.len

In [451]:
testSet = TriageTest(testData, tokenizer, MAX_LEN)

In [452]:
test_params = {'batch_size': 8,
                'shuffle': False,
                'num_workers': 0
                }


testing_loader = DataLoader(testSet, **test_params)

In [453]:
predictions = []
with torch.no_grad():
    for _, data in tqdm(enumerate(testing_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        outputs = model(ids, mask).squeeze()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        predictions.append(big_idx)



1715it [03:58,  7.19it/s]


In [454]:
print(predictions)
listOfList = [list(x.cpu().numpy()) for x in predictions]
result = [item for sublist in listOfList for item in sublist]

[tensor([16,  8, 10,  5,  5, 15,  5,  5], device='cuda:0'), tensor([ 8,  8, 16,  0,  9,  0, 16, 10], device='cuda:0'), tensor([16, 16,  4, 16, 16, 16, 19, 19], device='cuda:0'), tensor([11,  8,  8, 16, 16, 19, 16, 16], device='cuda:0'), tensor([ 3, 16, 16, 16,  8, 19, 10, 19], device='cuda:0'), tensor([ 7, 16,  8,  8, 14, 16,  8, 16], device='cuda:0'), tensor([ 6,  7,  5, 16, 16,  4, 13, 17], device='cuda:0'), tensor([19,  8,  4,  3, 16, 16, 19, 16], device='cuda:0'), tensor([16,  4, 16,  2, 16,  5,  2, 16], device='cuda:0'), tensor([ 2,  9, 16,  4,  4, 10, 19, 10], device='cuda:0'), tensor([10, 16, 19, 16, 16,  9,  4, 16], device='cuda:0'), tensor([19, 16,  2, 13,  3,  8, 16,  3], device='cuda:0'), tensor([16,  5, 19,  4, 16, 16,  8, 16], device='cuda:0'), tensor([16, 10,  4,  2, 16,  4,  6, 16], device='cuda:0'), tensor([ 2, 16, 16, 16, 16, 10, 16,  8], device='cuda:0'), tensor([ 2, 16, 16, 16, 16, 16, 16,  5], device='cuda:0'), tensor([ 5, 16, 10,  5, 16, 14,  5, 16], device='cuda:0

In [457]:
len(result)
result

[16,
 8,
 10,
 5,
 5,
 15,
 5,
 5,
 8,
 8,
 16,
 0,
 9,
 0,
 16,
 10,
 16,
 16,
 4,
 16,
 16,
 16,
 19,
 19,
 11,
 8,
 8,
 16,
 16,
 19,
 16,
 16,
 3,
 16,
 16,
 16,
 8,
 19,
 10,
 19,
 7,
 16,
 8,
 8,
 14,
 16,
 8,
 16,
 6,
 7,
 5,
 16,
 16,
 4,
 13,
 17,
 19,
 8,
 4,
 3,
 16,
 16,
 19,
 16,
 16,
 4,
 16,
 2,
 16,
 5,
 2,
 16,
 2,
 9,
 16,
 4,
 4,
 10,
 19,
 10,
 10,
 16,
 19,
 16,
 16,
 9,
 4,
 16,
 19,
 16,
 2,
 13,
 3,
 8,
 16,
 3,
 16,
 5,
 19,
 4,
 16,
 16,
 8,
 16,
 16,
 10,
 4,
 2,
 16,
 4,
 6,
 16,
 2,
 16,
 16,
 16,
 16,
 10,
 16,
 8,
 2,
 16,
 16,
 16,
 16,
 16,
 16,
 5,
 5,
 16,
 10,
 5,
 16,
 14,
 5,
 16,
 8,
 3,
 16,
 5,
 16,
 16,
 15,
 16,
 2,
 10,
 16,
 16,
 16,
 16,
 16,
 18,
 0,
 18,
 16,
 8,
 2,
 10,
 10,
 16,
 8,
 16,
 0,
 16,
 16,
 16,
 16,
 16,
 16,
 5,
 8,
 8,
 8,
 5,
 3,
 16,
 5,
 16,
 16,
 16,
 16,
 16,
 16,
 7,
 16,
 16,
 16,
 16,
 13,
 3,
 16,
 8,
 4,
 16,
 16,
 16,
 19,
 16,
 2,
 16,
 16,
 16,
 16,
 16,
 8,
 16,
 8,
 19,
 16,
 16,
 4,
 16,
 6,
 19,
 9,
 16,


In [458]:
submission = sample
submission['label'] = result

In [459]:
submission

,id,label
0,137832,16
1,137833,8
2,137834,10
3,137836,5
4,137837,5
...,...,...
13713,169336,16
13714,169338,16
13715,169340,10
13716,169341,16


In [460]:
submission.to_csv('submission1.csv', index = False)

In [ ]:
## list of things left to experiment and fine tune
# 1. train bert with abstract only
# 2. train bert with title and abstract
# 3. train bert with batch size at 16 or 32
# 4. Tweak the max length for padding depending on the distribution length of the paragraphs
## 5. Very important: also try with 4 epochs
## Play with the drop out level at 0.2, 0.3, and 0.5
## try scibert pre trained scientific model 
### SUPER IMPORTANT NEXT --> Make the first letter of the title upper case
### Super important too --> >>> import re
## >>> s = '<@ """@$ FSDF >something something <more noise>'
## >>> re.sub('<[^>]+>', '', s)
# 'something something ' For text that is delimited by $ $ and only if length between $ $ signs is greater than 3 Indeed for instance, we want to transform this $k$ into k sometimes
## Things between $ signs -- > $ $
## Things between # signs -- > # #
## in particular this delimiter #R##N#
## Things between â€ and â€ or â€
## remove special characters and try it on  witith >>> string = "Special $#! characters   spaces 888323"
## >>> ''.join(e for e in string if e.isalnum())
## â€™ bad characters like that --> remove specific instances like that â€
## also strings that start like that â€” and finish like that
## #R##N# --> bad characters like that
## remove anything that is between \begin{equation*}  and \end{equation*} example: this whole thing should be removed: \begin{equation*} u(f) = \frac{\lambda}{2}\Vert f\Vert_{H}^{2} +\frac{1}{2} \frac{1}{n}\sum_{i=1}^{n}\Vert f(x_i)-y_i\Vert_{V}^{2}. \end{equation*}
## make sure that things like these  {\it binary fused compressive sensing} (BFCS) go into  "binary fused compressive sensing" like that
## make sure you remove  {\em gossip} becomes gossip



In [ ]:
# Example bad script
## Twenty years ago, Bobkov, Houdr\'e, and the last author introduced a Poincar\'e-type functional graph parameter, $\lambda_\infty(G)$, of a graph $G$, and related it to the {\em vertex expansion} of $G$ via a Cheeger-type inequality. This is analogous to the Cheeger-type inequality relating the spectral gap, $\lambda_2(G)$, of the graph to its {\em edge expansion}.  While $\lambda_2$ can be computed efficiently, the computational complexity of $\lambda_\infty$ has remained an open question. Following the work of the second author with Raghavendra and Vempala, wherein the complexity of $\lambda_\infty$ was related to the so-called Small-Set Expansion (SSE) problem, it has been believed that computing $\lambda_\infty$ is a hard problem. We settle this question by proving that computing $\lambda_\infty$ is indeed NP-hard. Additionally, we use our techniques to prove NP-hardness of computing the spread constant (of a graph), a geometric measure introduced by Alon, Boppana, and Spencer, in the context of deriving an asymptotic isoperimetric inequality on cartesian products of graphs.  We complement our hardness results by providing approximation schemes for computing $\lambda_\infty$ and the spread constant of star graphs, and investigate constant approximability for weighted trees.  Finally, we provide improved approximation results for the maximum variance embedding problem for general graphs, by replacing the optimal orthogonal projection (PCA) with a randomized projection approach.

# Another example bad script
## The paper tackles the issue of $\textit{checking}$ that all copies of a large data set replicated at several nodes of a network are identical. The fact that the replicas may be located at distant nodes prevents the system from verifying their equality locally, i.e., by having each node consult only nodes in its vicinity. On the other hand, it remains possible to assign $\textit{certificates}$ to the nodes, so that verifying the consistency of the replicas can be achieved locally. However, we show that, as the data set is large, classical certification mechanisms, including distributed Merlin-Arthur protocols, cannot guarantee good completeness and soundness simultaneously, unless they use very large certificates. The main result of this paper is a distributed $\textit{quantum}$ Merlin-Arthur protocol enabling the nodes to collectively check the consistency of the replicas, based on small certificates, and in a single round of message exchange between neighbors, with short messages. In particular, the certificate-size is logarithmic in the size of the data set, which gives an exponential advantage over classical certification mechanisms.

## Another example of bad script
## In this paper we present CALVIS, a method to calculate $\textbf{C}$hest, w$\textbf{A}$ist and pe$\textbf{LVIS}$ circumference from 3D human body meshes. Our motivation is to use this data as ground truth for training convolutional neural networks (CNN). Previous work had used the large scale CAESAR dataset or determined these anthropometrical measurements $\textit{manually}$ from a person or human 3D body meshes. Unfortunately, acquiring these data is a cost and time consuming endeavor. In contrast, our method can be used on 3D meshes automatically. We synthesize eight human body meshes and apply CALVIS to calculate chest, waist and pelvis circumference. We evaluate the results qualitatively and observe that the measurements can indeed be used to estimate the shape of a person. We then asses the plausibility of our approach by generating ground truth with CALVIS to train a small CNN. After having trained the network with our data, we achieve competitive validation error. Furthermore, we make the implementation of CALVIS publicly available to advance the field.

## Another example of bad script
## The celebrated minimax principle of Yao (1977) says that for any Boolean-valued function $f$ with finite domain, there is a distribution $\mu$ over the domain of $f$ such that computing $f$ to error $\epsilon$ against inputs from $\mu$ is just as hard as computing $f$ to error $\epsilon$ on worst-case inputs. Notably, however, the distribution $\mu$ depends on the target error level $\epsilon$: the hard distribution which is tight for bounded error might be trivial to solve to small bias, and the hard distribution which is tight for a small bias level might be far from tight for bounded error levels. #R##N#In this work, we introduce a new type of minimax theorem which can provide a hard distribution $\mu$ that works for all bias levels at once. We show that this works for randomized query complexity, randomized communication complexity, some randomized circuit models, quantum query and communication complexities, approximate polynomial degree, and approximate logrank. We also prove an improved version of Impagliazzo's hardcore lemma. #R##N#Our proofs rely on two innovations over the classical approach of using Von Neumann's minimax theorem or linear programming duality. First, we use Sion's minimax theorem to prove a minimax theorem for ratios of bilinear functions representing the cost and score of algorithms. #R##N#Second, we introduce a new way to analyze low-bias randomized algorithms by viewing them as "forecasting algorithms" evaluated by a proper scoring rule. The expected score of the forecasting version of a randomized algorithm appears to be a more fine-grained way of analyzing the bias of the algorithm. We show that such expected scores have many elegant mathematical properties: for example, they can be amplified linearly instead of quadratically. We anticipate forecasting algorithms will find use in future work in which a fine-grained analysis of small-bias algorithms is required.

## Another example of bad script

## The class $FORMULA[s] \circ \mathcal{G}$ consists of Boolean functions computable by size-$s$ de Morgan formulas whose leaves are any Boolean functions from a class $\mathcal{G}$. We give lower bounds and (SAT, Learning, and PRG) algorithms for $FORMULA[n^{1.99}]\circ \mathcal{G}$, for classes $\mathcal{G}$ of functions with low communication complexity. Let $R^{(k)}(\mathcal{G})$ be the maximum $k$-party NOF randomized communication complexity of $\mathcal{G}$. We show: #R##N#(1) The Generalized Inner Product function $GIP^k_n$ cannot be computed in $FORMULA[s]\circ \mathcal{G}$ on more than $1/2+\varepsilon$ fraction of inputs for $$ s = o \! \left ( \frac{n^2}{ \left(k \cdot 4^k \cdot {R}^{(k)}(\mathcal{G}) \cdot \log (n/\varepsilon) \cdot \log(1/\varepsilon) \right)^{2}} \right).$$ As a corollary, we get an average-case lower bound for $GIP^k_n$ against $FORMULA[n^{1.99}]\circ PTF^{k-1}$. #R##N#(2) There is a PRG of seed length $n/2 + O\left(\sqrt{s} \cdot R^{(2)}(\mathcal{G}) \cdot\log(s/\varepsilon) \cdot \log (1/\varepsilon) \right)$ that $\varepsilon$-fools $FORMULA[s] \circ \mathcal{G}$. For $FORMULA[s] \circ LTF$, we get the better seed length $O\left(n^{1/2}\cdot s^{1/4}\cdot \log(n)\cdot \log(n/\varepsilon)\right)$. This gives the first non-trivial PRG (with seed length $o(n)$) for intersections of $n$ half-spaces in the regime where $\varepsilon \leq 1/n$. #R##N#(3) There is a randomized $2^{n-t}$-time $\#$SAT algorithm for $FORMULA[s] \circ \mathcal{G}$, where $$t=\Omega\left(\frac{n}{\sqrt{s}\cdot\log^2(s)\cdot R^{(2)}(\mathcal{G})}\right)^{1/2}.$$ In particular, this implies a nontrivial #SAT algorithm for $FORMULA[n^{1.99}]\circ LTF$. #R##N#(4) The Minimum Circuit Size Problem is not in $FORMULA[n^{1.99}]\circ XOR$. On the algorithmic side, we show that $FORMULA[n^{1.99}] \circ XOR$ can be PAC-learned in time $2^{O(n/\log n)}$.
## The orthogonality dimension of a graph $G=(V,E)$ over a field $\mathbb{F}$ is the smallest integer $t$ for which there exists an assignment of a vector $u_v \in \mathbb{F}^t$ with $\langle u_v,u_v \rangle \neq 0$ to every vertex $v \in V$, such that $\langle u_v, u_{v'} \rangle = 0$ whenever $v$ and $v'$ are adjacent vertices in $G$. The study of the orthogonality dimension of graphs is motivated by various application in information theory and in theoretical computer science. The contribution of the present work is two-folded. #R##N#First, we prove that there exists a constant $c$ such that for every sufficiently large integer $t$, it is $\mathsf{NP}$-hard to decide whether the orthogonality dimension of an input graph over $\mathbb{R}$ is at most $t$ or at least $3t/2-c$. At the heart of the proof lies a geometric result, which might be of independent interest, on a generalization of the orthogonality dimension parameter for the family of Kneser graphs, analogously to a long-standing conjecture of Stahl (J. Comb. Theo. Ser. B, 1976). #R##N#Second, we study the smallest possible orthogonality dimension over finite fields of the complement of graphs that do not contain certain fixed subgraphs. In particular, we provide an explicit construction of triangle-free $n$-vertex graphs whose complement has orthogonality dimension over the binary field at most $n^{1-\delta}$ for some constant $\delta >0$. Our results involve constructions from the family of generalized Kneser graphs and they are motivated by the rigidity approach to circuit lower bounds. We use them to answer a couple of questions raised by Codenotti, Pudlak, and Resta (Theor. Comput. Sci., 2000), and in particular, to disprove their Odd Alternating Cycle Conjecture over every finite field.

## In this paper a proof system is developed for plan verification problems $\{X\}c\{Y\}$ and $\{X\}c\{KW p\}$ under 0-approximation semantics for ${\mathcal A}_K$. Here, for a plan $c$, two sets $X,Y$ of fluent literals, and a literal $p$, $\{X\}c\{Y\}$ (resp. $\{X\}c\{KW p\}$) means that all literals of $Y$ become true (resp. $p$ becomes known) after executing $c$ in any initial state in which all literals in $X$ are true.Then, soundness and completeness are proved. The proof system allows verifying plans and generating plans as well.

## This paper considers the \textit{minimum spanning tree (MST)} problem in the Congested Clique model and presents an algorithm that runs in $O(\log \log \log n)$ rounds, with high probability. Prior to this, the fastest MST algorithm in this model was a deterministic algorithm due to Lotker et al.~(SIAM J on Comp, 2005) from about a decade ago. A key step along the way to designing this MST algorithm is a \textit{connectivity verification} algorithm that not only runs in $O(\log \log \log n)$ rounds with high probability, but also has low message complexity. This allows the fast computation of an MST by running multiple instances of the connectivity verification algorithm in parallel. These results depend on a new edge-sampling theorem, developed in the paper, that says that if each edge $e = \{u, v\}$ is sampled independently with probability $c \log^2 n/\min\{\mbox{degree}(u), \mbox{degree}(v)\}$ (for a large enough constant $c$) then all cuts of size at least $n$ are approximated in the sampled graph. This sampling theorem is inspired by series of papers on graph sparsification via random edge sampling due to Karger~(STOC 1994), Bencz\'ur and Karger~(STOC 1996, arxiv 2002), and Fung et al.~(STOC 2011). The edge sampling techniques in these papers use probabilities that are functions of edge-connectivity or a related measure called edge-strength. For the purposes of this paper, these edge-connectivity measures seem too costly to compute and the main technical contribution of this paper is to show that degree-based edge-sampling suffices to approximate large cuts.

## his paper looks at Turingâ€™s postulations about Artificial Intelligence in his paper â€˜Computing Machinery and Intelligenceâ€™, published in 1950. It notes how accurate they were and how relevant they still are today. This paper notes the arguments and mechanisms that he suggested and tries to expand on them further. The paper however is mostly about describing the essential ingredients for building an intelligent model and the problems related with that. The discussion includes recent work by the author himself, who adds his own thoughts on the matter that come from a purely technical investigation into the problem. These are personal and quite speculative, but provide an interesting insight into the mechanisms that might be used for building an intelligent system.

## A sumtest for a discrete semimeasure $P$ is a function $f$ mapping bitstrings to non-negative rational numbers such that \[ #R##N#\sum P(x)f(x) \le 1 \,. #R##N#\] Sumtests are the discrete analogue of Martin-L\"of tests. The behavior of sumtests for computable $P$ seems well understood, but for some applications lower semicomputable $P$ seem more appropriate. In the case of tests for independence, it is natural to consider upper semicomputable tests (see [B.Bauwens and S.Terwijn, Theory of Computing Systems 48.2 (2011): 247-268]). #R##N#In this paper, we characterize upper semicomputable sumtests relative to any lower semicomputable semimeasures using Kolmogorov complexity. It is studied to what extend such tests are pathological: can upper semicomputable sumtests for $m(x)$ be large? It is shown that the logarithm of such tests does not exceed $\log |x| + O(\log^{(2)} |x|)$ (where $|x|$ denotes the length of $x$ and $\log^{(2)} = \log\log$) and that this bound is tight, i.e. there is a test whose logarithm exceeds $\log |x| - O(\log^{(2)} |x|$) infinitely often. Finally, it is shown that for each such test $e$ the mutual information of a string with the Halting problem is at least $\log e(x)-O(1)$; thus $e$ can only be large for ``exotic'' strings.

## â€ â€ â€  â€ â€  â€ â€ â€  â€  â€  â€ â€  Summary Despite the evolution in deployed infrastructure and in the way that people access information, still there are those who are socially excluded and have no access to information due to their geographic location (e.g., riverside/countryside communities). This paper proposes an extension to a DTN architecture implementation to allow the dissemination of information in such communities, including educational short-video clips and audio books. The IBR-DTN architecture is complemented with a Bluetooth Convergence Layer, to facilitate the exchange of information over this short-range wireless technology, and with a Bundle Compression mechanism that aims at improving data exchange in short-lived opportunistic contacts happening among nodes. Experiments in a small-scale testbed and in a large-scale simulator environment show that nodes are indeed able to efficiently use contact opportunities to exchange an increased amount of data, allowing people in riverside communities to receive more content related to digital inclusion services.